In [73]:
# 载入one-hot数据


import mxnet as mx
import numpy as np

# def load_one_hot_data():
data_list = []
print '>> loading ... ',
for line in open('/home01/shixiangwan/deep_learning/protein_location/RNA-all-long-CD-HIT.fasta'):
    if line[0] != '>':
        line = line.strip().replace('A', '1').replace('U', '2').replace('C', '3').replace('G', '4')
        data_list.append(map(int, list(line)))
depth = len(max(data_list, key=len))
print 'finished.'


>> loading ...  finished.


/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 17:24:05.439345. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [82]:
import time

data_mx = []
count = 0
start = time.clock()
for elem in range(len(data_list)):
    print 'data_list length:', len(data_list[elem])
    tmp_mx = mx.nd.one_hot(mx.nd.array(data_list[elem], dtype=np.int32), depth=depth)\
        .asnumpy().reshape(len(data_list[elem]) * depth)
    tmp_mx = np.append(tmp_mx, np.zeros(len(data_list[elem]) * depth))
    data_mx.append(list(tmp_mx))
    count += 1
    print count
    if count == 2:
        break

print time.clock() - start, 'ms'


data_list length: 7091
0.210196
0.561533
1
data_list length: 1484
0.0377910000002
0.132156
2


/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 18:57:32.208196. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [85]:
# %prun load_one_hot_data()

# data_mx = load_one_hot_data()
one_hot = open('/home01/shixiangwan/deep_learning/protein_location/one_hot.csv', 'w')
for i in range(len(data_mx)):
    one_hot.write(','.join(map(str, data_mx[i]))+'\n')
one_hot.close()

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 19:00:03.626092. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [10]:
# 载入数据
def load_data():
    train_set = []
    label_set = []
    for line in open("RNA-all-SC-PseDNC-General.csv"):
        a_line = ''
        for i in range(34):
            a_line += line + ','
        train_set.append(map(float, a_line.strip(',').split(',')))
    for line in open("labels_1.txt"):
#         label_set.append(map(float, line.strip().split(',')))
        label_set.append(float(line.strip()))
    # 提取二分类平衡数据集
    lbl_0 = label_set.count(0.0)
    lbl_1 = label_set.count(1.0)
    lbl_less = 1
    lbl_less_num = lbl_1
    if max(lbl_0, lbl_1) == lbl_1:
        lbl_less = 0
        lbl_less_num = lbl_0
    train_set2 = []
    label_set2 = []
    count_more = 0
    for i in range(len(label_set)):
        if label_set[i] == lbl_less:
            train_set2.append(train_set[i])
            label_set2.append(label_set[i])
        else:
            if count_more < lbl_less_num:
                train_set2.append(train_set[i])
                label_set2.append(label_set[i])
                count_more += 1
    print lbl_less, lbl_less_num
    return train_set2, label_set2

# train_set2, label_set2 = load_data()
# print len(train_set2)
# print len(label_set2)


0 1218
2436
2436


In [38]:
# %load protein.py

import logging

import mxnet as mx
import numpy as np

import random


# 数据标准化处理
def norm_stat(d):
    return mx.nd.norm(d) / np.sqrt(d.size)


# 多层感知机
def set_mlp():
    data = mx.symbol.Variable('data')
    data = mx.sym.Flatten(data=data)
    fc1 = mx.symbol.FullyConnected(data=data, name='fc1', num_hidden=128)
    act1 = mx.symbol.Activation(data=fc1, name='relu1', act_type="relu")
    fc2 = mx.symbol.FullyConnected(data=act1, name='fc2', num_hidden=64)
    act2 = mx.symbol.Activation(data=fc2, name='relu2', act_type="relu")
    fc3 = mx.symbol.FullyConnected(data=act2, name='fc3', num_hidden=3)
    mlp = mx.symbol.SoftmaxOutput(data=fc3, name='softmax')
    return mlp


# Basic Conv + BN + ReLU factory
def ConvFactory(data, num_filter, kernel, stride=(1,1), pad=(0, 0), act_type="relu"):
    # there is an optional parameter ```wrokshpace``` may influece convolution performance
    # default, the workspace is set to 256(MB)
    # you may set larger value, but convolution layer only requires its needed but not exactly
    # MXNet will handle reuse of workspace without parallelism conflict
    conv = mx.symbol.Convolution(data=data, workspace=256,
                                 num_filter=num_filter, kernel=kernel, stride=stride, pad=pad)
    bn = mx.symbol.BatchNorm(data=conv)
    act = mx.symbol.Activation(data = bn, act_type=act_type)
    return act


# A Simple Downsampling Factory
def DownsampleFactory(data, ch_3x3):
    # conv 3x3
    conv = ConvFactory(data=data, kernel=(3, 3), stride=(2, 2), num_filter=ch_3x3, pad=(1, 1))
    # pool
    pool = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(2, 2), pad=(1,1), pool_type='max')
    # concat
    concat = mx.symbol.Concat(*[conv, pool])
    return concat


# A Simple module
def SimpleFactory(data, ch_1x1, ch_3x3):
    # 1x1
    conv1x1 = ConvFactory(data=data, kernel=(1, 1), pad=(0, 0), num_filter=ch_1x1)
    # 3x3
    conv3x3 = ConvFactory(data=data, kernel=(3, 3), pad=(1, 1), num_filter=ch_3x3)
    #concat
    concat = mx.symbol.Concat(*[conv1x1, conv3x3])
    return concat


def set_cifar10():
    data = mx.symbol.Variable(name="data")
    conv1 = ConvFactory(data=data, kernel=(3,3), pad=(1,1), num_filter=96, act_type="relu")
    in3a = SimpleFactory(conv1, 32, 32)
    in3b = SimpleFactory(in3a, 32, 48)
    in3c = DownsampleFactory(in3b, 80)
    in4a = SimpleFactory(in3c, 112, 48)
    in4b = SimpleFactory(in4a, 96, 64)
    in4c = SimpleFactory(in4b, 80, 80)
    in4d = SimpleFactory(in4c, 48, 96)
    in4e = DownsampleFactory(in4d, 96)
    in5a = SimpleFactory(in4e, 176, 160)
    in5b = SimpleFactory(in5a, 176, 160)
    pool = mx.symbol.Pooling(data=in5b, pool_type="avg", kernel=(7,7), name="global_avg")
    flatten = mx.symbol.Flatten(data=pool)
    fc = mx.symbol.FullyConnected(data=flatten, num_hidden=10)
    softmax = mx.symbol.SoftmaxOutput(name='softmax',data=fc)
    return softmax


# 卷积神经网络
def set_con():
    data = mx.symbol.Variable('data')
    # first conv layer
    mx.sym.Activation
    conv1 = mx.sym.Convolution(data=data, kernel=(5, 5), num_filter=20)
    tanh1 = mx.sym.Activation(data=conv1, act_type="tanh")
    pool1 = mx.sym.Pooling(data=tanh1, pool_type="max", kernel=(2, 2), stride=(2, 2))
    # second conv layer
    conv2 = mx.sym.Convolution(data=pool1, kernel=(5, 5), num_filter=50)
    tanh2 = mx.sym.Activation(data=conv2, act_type="tanh")
    pool2 = mx.sym.Pooling(data=tanh2, pool_type="max", kernel=(2, 2), stride=(2, 2))
    # first fullc layer
    flatten = mx.sym.Flatten(data=pool2)
    fc1 = mx.symbol.FullyConnected(data=flatten, num_hidden=500)
    tanh3 = mx.sym.Activation(data=fc1, act_type="tanh")
    # second fullc
    fc2 = mx.sym.FullyConnected(data=tanh3, num_hidden=2)
    # softmax loss
    lenet = mx.sym.SoftmaxOutput(data=fc2, name='softmax')
    return lenet


def to4d(data):
    print data.shape
    return data.reshape(data.shape[0], 1, 34, 34)


if __name__ == '__main__':
    # data
    train_set, label_set = load_data()
    all_number = len(label_set)
    split_tv = int(0.8*all_number)
    split_tt = int(0.9*all_number)
    train_iter = mx.io.NDArrayIter(mx.nd.array(to4d(np.array(train_set[0:split_tv]))),
                                  mx.nd.array(np.array(label_set[0:split_tv])),
                                  shuffle=True)
    validate_iter = mx.io.NDArrayIter(mx.nd.array(to4d(np.array(train_set[split_tv:split_tt]))),
                                   mx.nd.array(np.array(label_set[split_tv:split_tt])),
                                   shuffle=True)

    # train
    logging.basicConfig(level=logging.INFO)
    model = mx.model.FeedForward(ctx=mx.gpu(1), # [mx.gpu(i) for i in range(4)]
                                 symbol=set_con(), # set_mlp(), set_con(), set_cifar10()
                                 num_epoch=1,
                                 learning_rate=0.1,
                                 momentum=0.9,
                                 wd=0.00001)

    batch_size = 100
    result = model.fit(X = train_iter,
                       eval_data = validate_iter,
                       batch_end_callback = mx.callback.Speedometer(batch_size, 200)
    )
    

/home/shixiangwan/.local/lib/python2.7/site-packages/ipykernel/__main__.py:132: DeprecationWarning: mxnet.model.FeedForward has been deprecated. Please use mxnet.mod.Module instead.
INFO:root:Start training with [gpu(1)]


0 1218
(1948, 1156)
(244, 1156)


INFO:root:Epoch[0] Batch [200]	Speed: 63934.15 samples/sec	Train-accuracy=0.530000
INFO:root:Epoch[0] Batch [400]	Speed: 71173.07 samples/sec	Train-accuracy=0.535000
INFO:root:Epoch[0] Batch [600]	Speed: 71126.55 samples/sec	Train-accuracy=0.530000
INFO:root:Epoch[0] Batch [800]	Speed: 71774.62 samples/sec	Train-accuracy=0.545000
INFO:root:Epoch[0] Batch [1000]	Speed: 70561.88 samples/sec	Train-accuracy=0.555000
INFO:root:Epoch[0] Batch [1200]	Speed: 68987.65 samples/sec	Train-accuracy=0.495000
INFO:root:Epoch[0] Batch [1400]	Speed: 70990.45 samples/sec	Train-accuracy=0.540000
INFO:root:Epoch[0] Batch [1600]	Speed: 71118.95 samples/sec	Train-accuracy=0.525000
INFO:root:Epoch[0] Batch [1800]	Speed: 69715.32 samples/sec	Train-accuracy=0.485000
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=2.805
INFO:root:Epoch[0] Validation-accuracy=1.000000
/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 12:02:00.95861

In [40]:
# 分析结果

test_set = train_set[split_tt:]
test_lbl = label_set[split_tt:]

count = 0
r_num = 0
results = open('results.csv', 'w')
for i in range(len(test_set)):
    test_val = np.array(test_set[i]).reshape(1, 1, 34, 34)
    prob = model.predict(test_val)
    print prob[0]
    predict =  ','.join(str(j) for j in prob[0])
    # origin = ','.join(str(j) for j in test_lbl[i])
    origin = str(test_lbl[i])
    for j in range(len(prob[0])):
        p_val = 0
        if prob[0][j] >= 0.5:
            p_val = 1
        # if p_val == test_lbl[i][j]:
        if p_val == test_lbl[i]:
            r_num += 1
    results.write(predict + '\n' + origin + '\n\n')
    count += 1
#     print predict
#     print origin
#     print
#     if count == 20:
#         break
results.close()

# performance matrics
test_iter = mx.io.NDArrayIter(mx.nd.array(to4d(np.array(test_set))),
                              mx.nd.array(np.array(test_lbl)),
                              shuffle=True)
f1, mse, acc = model.score(test_iter, ['f1', 'mse', 'acc'])
print '----------------------------------------------------------------------'
print 'precision\t\t' + 'mse'
print acc, '\t\t', mse, '\t\t', f1

# print '\nright/all/rate:', r_num, count*len(label_set[0]), float(r_num)/(count*len(label_set[0]))
print '\nright/all/rate:', r_num, count, float(r_num)/(count)

[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.54688876e-36]
[  1.00000000e+00   1.546888

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 12:11:02.198520. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


推测：

在更复杂的蛋白质二级结构预测方面，卷积神经网络应该有着更好的效果。

In [60]:
# mx.nd.one_hot(mx.nd.array(np.array([1, 0, 2, 0]), dtype=np.int32), depth=5).asnumpy()
mx.nd.one_hot(mx.nd.array([1, 2, 3, 4, 1], dtype=np.int32), depth=10).asnumpy()

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

/usr/lib/python2.7/json/encoder.py:207: DeprecationWarning: Interpreting naive datetime as local 2017-03-17 15:57:12.689383. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
